In [ ]:
#pip install llama-index-llms-huggingface

  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_workflows-1.1.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wrapt-1.17.2-cp310-cp310-win_amd64.whl.metadata (6.5 kB)
  Using cached griffe-1.7.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached llama_index_instrumentation-0.3.0-py3-none-any.whl.metadata (252 bytes)
  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/7.6 MB 8.4 MB/s eta 0:00:01
   ------------------- -------------------- 3.7/7.6 MB 10.4 MB/s eta 0:00:01
   --------------

In [ ]:
#pip install llama-index

  Using cached llama_index_agent_openai-0.4.12-py3-none-any.whl.metadata (439 bytes)
  Using cached llama_index_cli-0.4.4-py3-none-any.whl.metadata (1.4 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.7.10-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_llms_openai-0.4.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached llama_index_multi_modal_llms_openai-0.5.3-py3-none-any.whl.metadata (441 bytes)
  Using cached llama_index_program_openai-0.3.2-py3-none-any.whl.metadata (473 bytes)
  Using cached llama_index_question_gen_openai-0.3.1-py3-none-any.whl.metadata (492 bytes)
  Using cached llama_index_readers_file-0.4.11-py3-none-any.whl.metadata (5.3 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (

In [ ]:
# pip install llama-index-embeddings-huggingface

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install llama-index-embedding-instructor

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement llama-index-embedding-instructor (from versions: none)
ERROR: No matching distribution found for llama-index-embedding-instructor


In [17]:
pip install llama-index-llms-huggingface

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install llama-index

Note: you may need to restart the kernel to use updated packages.


In [1]:
from llama_index.core import SimpleDirectoryReader

directory_reader = SimpleDirectoryReader(input_dir='D:\RAG\RAG App using LLamaindex and Mistral')
directory_reader.load_data()

[Document(id_='2aabbaf9-2df1-4b0b-ae7e-f3ba64e96115', embedding=None, metadata={'page_label': '1', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': 'D:\\RAG\\RAG App using LLamaindex and Mistral\\MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482143, 'creation_date': '2025-07-22', 'last_modified_date': '2025-07-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/355917108\nNeural Machine Translation with Attention\nTechnical Report · August 2021\nDOI: 10.13140/RG.2.2.29381.37607

In [2]:
from llama_index.core import PromptTemplate

system_prompt = """ <|SYSTEM|># You are a QA Assistant. Your goal is to answer question accurately
as possible based on the context and your knowledge.
"""
query_wrapper_prompts = PromptTemplate("<|USER|>{query}<|ASSISTANT|>")

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
from huggingface_hub import login

login(os.getenv("HF_API_KEY"))


In [ ]:
# mistralai/Mistral-7B-v0.1

In [1]:
pip install hf_xet

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 2.1 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.7 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    query_wrapper_prompt=query_wrapper_prompts,
    system_prompt=system_prompt,
    tokenizer_name="mistralai/Mistral-7B-v0.1",
    model_name="mistralai/Mistral-7B-v0.1",
    device_map='auto',
    stopping_ids=[],
    tokenizer_kwargs={'max_length':4096},
    model_kwargs={'torch_dtype':torch.float32},
    generate_kwargs={'temperature':.5,'do_sample':False}

)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

d:\RAG\rag_venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ajith\.cache\huggingface\hub\models--mistralai--Mistral-7B-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:
# from llama_index.llms.huggingface import HuggingFaceLLM
# import torch

# llm = HuggingFaceLLM(
#     context_window=4096,
#     max_new_tokens=256,
#     query_wrapper_prompt=query_wrapper_prompts,
#     system_prompt=system_prompt,
#     tokenizer_name="mistralai/Mistral-Nemo-Base-2407",
#     model_name="mistralai/Mistral-Nemo-Base-2407",
#     device_map='auto',
#     stopping_ids=[],
#     tokenizer_kwargs={'max_length':4096},
#     model_kwargs={'torch_dtype':torch.float32},
#     generate_kwargs={'temperature':.5,'do_sample':False}

# )

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name = 'sentence-transformers/all-mpnet-base-v2',
)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

In [ ]:
# Settings.llm = OpenAI(model="gpt-3.5-turbo")
# Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
# Settings.num_output = 512
# Settings.context_window = 3900

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = node_parser
Settings.num_output = 512
Settings.context_window = 3900

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents( )